In [17]:
# Imports

import os
import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [18]:
import sys
print(sys.executable)

/Users/robinwarden/Desktop/Projects/job-copilot/job-copilot/bin/python


In [19]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [20]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Say hello to Robin!"}
]

In [21]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [22]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [23]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Hello to Robin too! I don't know who Robin is, though. Is it a friend or someone you'd like to talk about?


In [24]:
# 🏗️ Class to represent a job listing webpage

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Job_Platform:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # Remove unwanted tags
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)


In [25]:
# 💬 Build a user prompt from the job listing
def user_prompt_for(job_listing):
    user_prompt = f"You are looking at a job listing titled: {job_listing.title}\n"
    user_prompt += "The contents of this listing are as follows. Please provide a short summary in markdown format. \
If there are job requirements, duties, or application instructions, highlight them:\n\n"
    user_prompt += job_listing.text
    return user_prompt


In [26]:
# 🧠 Send the user/system prompt to OpenAI
def ask_ollama(system_prompt, user_prompt, model=MODEL):
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )
    return response['message']['content']


In [27]:
# 📂 Load resume from file
with open("resume.txt", "r") as file:
    my_resume = file.read()


In [28]:
print(my_resume[:500])  # Show the first 500 characters to make sure it loaded


 
Robin Warden
Software Engineer | Frontend & API Developer | AI Automation Enthusiast
Lawrenceville, GA       
(470) 406-1714     
Email| LinkedIn | GitHub | Portfolio

+Professional Summary

Creative and AI-native Frontend Developer with 3+ years of experience building scalable web and mobile interfaces using React, TypeScript, HTML/CSS, and modern CSS frameworks including Styled Components, Bootstrap, and Tailwind CSS. SME-level React engineer with a strong focus on component reusability, sca


In [31]:
# 📝 Define the system's behavior
system_prompt = (
    "You are a helpful job-matching assistant. "
    "Given a job listing and the candidate's resume, return the following:\n"
    "1. A match score from 0 to 100\n"
    "2. A short explanation of the score\n"
    "3. Highlight key overlapping skills or experience\n"
    "Be honest if the resume is not a strong fit."
)

user_prompt = f"""
## Job Description:
{job_page.text}

## Candidate Resume:
{my_resume}
"""




In [32]:
# 🔁 List of job board URLs to scrape
job_urls = [
    "https://www.dice.com/jobs?q=software+development&location=Georgia%2C+USA&latitude=32.1574351&longitude=-82.90712300000001&countryCode=US&locationPrecision=State&adminDistrictCode=GA&radiusUnit=mi",
    "https://www.dice.com/jobs?q=AI+Engineer&location=Georgia%2C+USA&latitude=32.1574351&longitude=-82.90712300000001&countryCode=US&locationPrecision=State&adminDistrictCode=GA&radiusUnit=mi",
    "https://www.dice.com/jobs?adminDistrictCode=GA&countryCode=US&latitude=32.1574351&location=Georgia%2C+USA&locationPrecision=State&longitude=-82.90712300000001&q=frontend&radiusUnit=mi",
]

# 💡 Optional: Store all summaries in a list for later use
job_summaries = []

# 🧠 Loop through each job URL
for idx, url in enumerate(job_urls, start=1):
    try:
        print(f"\n🔍 [{idx}] Scraping job listing from:\n{url}")

        # Scrape and process the page
        job_page = Job_Platform(url)
        user_prompt = user_prompt_for(job_page)

        # Ask Ollama for a summary
        summary = ask_ollama(system_prompt, user_prompt)

        # Print or collect the summary
        print(f"\n📝 Summary from {url}:\n")
        print(summary)

        # Optional: Save for later
        job_summaries.append({
            "url": url,
            "title": job_page.title,
            "summary": summary
        })

    except Exception as e:
        print(f"❌ Failed to process {url}: {e}")



🔍 [1] Scraping job listing from:
https://www.dice.com/jobs?q=software+development&location=Georgia%2C+USA&latitude=32.1574351&longitude=-82.90712300000001&countryCode=US&locationPrecision=State&adminDistrictCode=GA&radiusUnit=mi

📝 Summary from https://www.dice.com/jobs?q=software+development&location=Georgia%2C+USA&latitude=32.1574351&longitude=-82.90712300000001&countryCode=US&locationPrecision=State&adminDistrictCode=GA&radiusUnit=mi:

To match the candidate with the job, I'll need to see their resume. Please provide it, and I'll be happy to assist.

Once you share your resume, I can:

1. Match the skills and experience listed on the resume with the requirements of each job listing.
2. Calculate a score based on the matches (up to 100 points).
3. Provide an explanation for the score and highlight key overlapping skills or experiences.

Please paste your resume, and I'll get started!

🔍 [2] Scraping job listing from:
https://www.dice.com/jobs?q=AI+Engineer&location=Georgia%2C+USA&la